In [2]:
import pandas as pd
from stage import fullPrint, getIndustryStage
from datetime import datetime, timedelta
import numpy as np
import pickle
import os
import investpy
import swifter

PEAK = 12
allList = []
industryList = pd.read_pickle("stockData/industryList.pkl")

SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}

i = 0
now = datetime.now()

In [ ]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
WMA30SLOPE = 7
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,WMA30SLOPE]/df.iat[index-1,WMA30SLOPE]

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,0]/df.iat[index-30,0]
        
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,0]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,0])

In [ ]:
spDf = investpy.get_index_historical_data(index='S&P 500', country='United States', from_date='01/01/2010', to_date='01/01/2022', as_json=False, order='ascending', interval='Weekly')
spDf = spDf[~spDf.index.duplicated()]
spDf['Volume'] = spDf['Volume'].replace(to_replace=0, method='ffill')
spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf['WMA30'] = spDf['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
spDf['WMA30Slope'] = spDf.swifter.apply(lambda x: calculateSlopeOfTicker(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf = spDf.drop(['Open','High','Low','Currency'],axis=1)
spDf.to_pickle("stockData/Spy.pkl")

In [3]:
for industry in industryList:
    print(industry)
    try:
        stockDf = pd.read_pickle('stockData/industriesData/'+str(industry[0])+'/'+str(industry[1])+'.pkl')
        if stockDf['Close'].isnull().values.any():
            os.remove('stockData/industriesData/'+str(industry[0])+'/'+str(industry[1])+'.pkl')
        stockDf['fiveYearHigh'] = stockDf['Close'].rolling(window=260).max().shift(1).fillna(0)
        stockDf['WMA30'] = stockDf['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['Volume'] = stockDf['Volume'].replace(to_replace=0, method='ffill')
        stockDf['volumePerc'] = stockDf['Volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = 0
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, PEAK] = calculatePeak(stockDf, numIndex, 260, 3)
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        stockDf['resistance'] = 0
        stockDf['shortPeak'] = 0
        stockDf['shortTrough'] = 0
        stockDf = stockDf.drop(['Open','High','Low','Currency'],axis=1)
        stockDf.to_pickle('stockData/industriesData/'+str(industry[0])+'/'+str(industry[1])+'.pkl')
    except:
        pass

('XLP', 'DJTobacco')
('XLP', 'DJSoftDrinks')
('XLP', 'DJDrugRetailers')
('XLP', 'DJDistillers&Vintners')
('XLP', 'DJFoodProductMakers')
('XLP', 'DJFoodRetail&Wholesale')
('XLP', 'DJNondurableHousehold')
('XLP', 'DJBrewers')
('XLP', 'DJPersonalProducts')
('XLU', 'DJGasDistribution')
('XLU', 'DJMultiutilities')
('XLU', 'DJConventionalElectricity')
('XLU', 'DJWater')
('XLV', 'DJPharmaceuticals')
('XLV', 'DJMedicalSupplies')
('XLV', 'DJHealthCareProviders')
('XLV', 'DJMedicalEquipment')
('XLV', 'DJBiotechnology')
('XLF', 'DJFinancialAdministration')
('XLF', 'DJInsuranceBrokers')
('XLF', 'DJSpecialtyFinance')
('XLF', 'DJFullLineInsurance')
('XLF', 'DJConsumerFinance')
('XLF', 'DJAssetManagers')
('XLF', 'DJInvestmentServices')
('XLF', 'DJReinsurance')
('XLF', 'DJProperty&CasualtyInsurance')
('XLF', 'DJLifeInsurance')
('XLF', 'DJMortgageFinance')
('XLF', 'DJBanks')
('XLE', 'DJCoal')
('XLE', 'DJIntegratedOil&Gas')
('XLE', 'DJPipelines')
('XLE', 'DJExploration&Production')
('XLE', 'DJOilEquipme

In [ ]:
transactionTemplate = investpy.get_index_historical_data(index='DJ Commodity Chemicals',
                                        country='United States',
                                        from_date='01/01/2010',
                                        to_date='01/01/2022').drop(['Open','High','Low','Close','Volume','Currency'],axis=1)
transactionTemplate = transactionTemplate[~transactionTemplate.index.duplicated()]
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [5]:
fullPrint(pd.read_pickle('stockData/industriesData/'+str(industry[0])+'/'+str(industry[1])+'.pkl'))

              Close     Volume  fiveYearHigh        WMA30  WMA30Slope  \
Date                                                                    
2010-01-03   361.42  115989848          0.00          NaN    0.000000   
2010-01-10   350.91   83652968          0.00          NaN    0.000000   
2010-01-17   338.04   89958688          0.00          NaN    0.000000   
2010-01-24   341.96   80091616          0.00          NaN    0.000000   
2010-01-31   343.15  120258416          0.00          NaN    0.000000   
2010-02-07   346.31   79541712          0.00          NaN    0.000000   
2010-02-14   358.20   52057736          0.00          NaN    0.000000   
2010-02-21   370.08   88763440          0.00          NaN    0.000000   
2010-02-28   387.30   94989736          0.00          NaN    0.000000   
2010-03-07   394.83   90669544          0.00          NaN    0.000000   
2010-03-14   399.55   88271920          0.00          NaN    0.000000   
2010-03-21   408.76   72175968          0.00       

In [ ]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf.index = sectorDfList[0].index
index = 0
for date in sectorDfList[0].index:
    listOfSector = []
    for df in sectorDfList:
        try:
            if  df.at[date,'MA30Slope'] > 0.992549883242513 and df.loc[date,'RS'] > -0.3044455098712273:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    #print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
        except: 
            pass
        # print(listOfSector)
    goodSectorDf.iat[index,0] = set(listOfSector)
    index = index + 1


In [ ]:
removeList = []
CLOSE = 0
TICKER = 2
for ticker in nasdaqList:
    try:
        df = pd.read_pickle("stockData/nyseNasdaq/"+ticker+".pkl")
        lastClose = df.iat[0,CLOSE]
        isFirst = True
        index = 0
        for date in df.Stage:
            if isFirst == True:
                isFirst = False
                continue
            if (df.iat[index,CLOSE] >= lastClose * 5):
                removeList.append(ticker)
                break
            lastClose = df.iat[index,CLOSE]
            index += 1
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        buyDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        if stockDf['close'].isnull().values.any() or buyDf['open'].isnull().values.any():
            print(ticker)
            removeList.append(ticker)
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            os.remove('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        df = getStage(ticker,[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804],goodSectorDf)
        if df.empty:
            removeList.append(ticker)
    except:
        removeList.append(ticker)
print(removeList)

In [ ]:
for ticker in removeList:
    nasdaqList.remove(ticker)
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(nasdaqList, f)